In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [20]:
url = 'https://census2011.adrianfrith.com/'

sites=[]
sitelist=[]
subsitelist=[]

In [22]:
page = requests.get("https://census2011.adrianfrith.com/place/798")
apage = BeautifulSoup(page.text, 'lxml')

In [23]:
for tr in apage.find_all('tr'):
    aref = tr.find_all('a')
    for a in aref:
        sitelist.append(a.get('href'))

In [59]:
sitelist[:5]

['/place/798014',
 '/place/798011',
 '/place/798002',
 '/place/798012',
 '/place/798003']

In [24]:
for site in sitelist:
    bpage = requests.get(url+site)
    cpage = BeautifulSoup(bpage.text, 'lxml')
    for tr in cpage.find_all('tr'):
        aref = tr.find_all('a')
        for a in aref:
            subsitelist.append(a.get('href'))

In [60]:
subsitelist[:5]

['/place/798014056',
 '/place/798014048',
 '/place/798014053',
 '/place/798014061',
 '/place/798014045']

In [44]:
def myscraper(subsite):
    dpage = requests.get(url+subsite)
    mypage = BeautifulSoup(dpage.text, 'lxml')
    
    gender_name = []
    gender_num = []
    gender_loc = []
    gender_df = pd.DataFrame(columns=['locID', 'gender', 'count'])

    race_name = []
    race_num = []
    race_loc = []
    race_df = pd.DataFrame(columns=['locID', 'race', 'count'])

    lang_name = []
    lang_num = []
    lang_loc = []
    lang_df = pd.DataFrame(columns=['locID', 'language', 'count'])

    locat = mypage.select('.topname')[0].getText()
    
    try:
        # The tables are always in the same order: gender, race, language
        gender_table = mypage.find_all("table")[0]
        for td in gender_table.find_all("td"):
            if '%' not in td.text:
                try:
                    value = float(td.text)
                    gender_num.append(value)
                    gender_loc.append(locat)
                except:
                    gender_name.append(td.text)

        # Get the second table about race
        race_table = mypage.find_all("table")[1]
        for td in race_table.find_all("td"):
            if '%' not in td.text:
                try:
                    value = float(td.text)
                    race_num.append(value)
                    race_loc.append(locat)
                except:
                    race_name.append(td.text)

        language_table = mypage.find_all("table")[2]
        count = 0
        for td in language_table.find_all("td"):
            if (('%' not in td.text) and (td.text != "")):
                if count % 2 == 0:
                    lang_loc.append(locat)
                    lang_name.append(td.text)
                    count += 1
                else:
                    value = float(td.text)
                    lang_num.append(value)
                    count += 1

        gender_df['locID'] = gender_loc
        gender_df['gender'] = gender_name
        gender_df['count'] = gender_num

        race_df['locID'] = race_loc
        race_df['race'] = race_name
        race_df['count'] = race_num

        lang_df['locID'] = lang_loc
        lang_df['language'] = lang_name
        lang_df['count'] = lang_num
    
    except:
        pass
    
    return gender_df, race_df, lang_df

for subsite in subsitelist[:5]:
    gender, race, language = myscraper(subsite)

In [45]:
all_gender = []
all_race = []
all_lang = []
total = len(subsitelist)
count = 1
for subsite in subsitelist:
    gender, race, language = myscraper(subsite)
    all_gender.append(gender)
    all_race.append(race)
    all_lang.append(language)
    print(count, "complete of ", total)
    count+=1
    

1 complete of  804
2 complete of  804
3 complete of  804
4 complete of  804
5 complete of  804
6 complete of  804
7 complete of  804
8 complete of  804
9 complete of  804
10 complete of  804
11 complete of  804
12 complete of  804
13 complete of  804
14 complete of  804
15 complete of  804
16 complete of  804
17 complete of  804
18 complete of  804
19 complete of  804
20 complete of  804
21 complete of  804
22 complete of  804
23 complete of  804
24 complete of  804
25 complete of  804
26 complete of  804
27 complete of  804
28 complete of  804
29 complete of  804
30 complete of  804
31 complete of  804
32 complete of  804
33 complete of  804
34 complete of  804
35 complete of  804
36 complete of  804
37 complete of  804
38 complete of  804
39 complete of  804
40 complete of  804
41 complete of  804
42 complete of  804
43 complete of  804
44 complete of  804
45 complete of  804
46 complete of  804
47 complete of  804
48 complete of  804
49 complete of  804
50 complete of  804
51 comple

ConnectionError: ('Connection aborted.', OSError("(54, 'ECONNRESET')",))

In [46]:
gender_df = pd.concat(all_gender)
race_df = pd.concat(all_race)
lang_df = pd.concat(all_lang)

In [47]:
lang_df.head()

,locID,language,count
0,Alexandra Ext 1,Sepedi,1440.0
1,Alexandra Ext 1,isiZulu,1163.0
2,Alexandra Ext 1,Xitsonga,608.0
3,Alexandra Ext 1,Setswana,536.0
4,Alexandra Ext 1,Sesotho,434.0


In [55]:
gender = gender_df.pivot_table(index='locID', columns='gender', values='count').fillna(0)
race = race_df.pivot_table(index='locID', columns='race', values='count').fillna(0)
language = lang_df.pivot_table(index='locID', columns='language', values='count').fillna(0)

In [56]:
print(gender.head())
print(race.head())
print(language.head())

gender            Female    Male
locID                           
Abbotsford         218.0   170.0
Aeroton              1.0    10.0
Alan Manor        1233.0  1050.0
Alexandra Ext 1   2503.0  2764.0
Alexandra Ext 10   876.0   989.0
race              Black African  Coloured  Indian or Asian  Other  White
locID                                                                   
Abbotsford                139.0       7.0             17.0    2.0  223.0
Aeroton                    11.0       0.0              0.0    0.0    0.0
Alan Manor               1075.0     441.0            254.0    6.0  507.0
Alexandra Ext 1          5240.0      21.0              3.0    2.0    1.0
Alexandra Ext 10         1837.0       5.0              0.0   23.0    0.0
language          Afrikaans  English  Not applicable  Other  Sepedi  Sesotho  \
locID                                                                          
Abbotsford              4.0    262.0             0.0   11.0    13.0      5.0   
Aeroton           

In [58]:
gender.to_csv('csvs/jhb_gender.csv')
race.to_csv('csvs/jhb_race.csv')
language.to_csv('csvs/jhb_lang.csv')

In [ ]:
for subsite in subsitelist:
    mylink = url+subsite+"/kml"
    urllib.urlretrieve (mylink, "kmls/" + subsite + ".kml")